In [1]:
import requests
import pandas
from arcgis.features import GeoAccessor
from arcgis.geometry import SpatialReference
import zipfile

Load data from Minnesota Geospatial Commons

In [2]:
#Data url
mngeo_url = 'http://gisdata.mn.gov/api/3/action/package_show?id=bdry-counties-in-minnesota'

In [3]:
#Use CKAN API to call data from Minnesota Geospatial Commons
mngeo_response = requests.get(mngeo_url)

In [4]:
#View the response as a json
mngeo_data = mngeo_response.json()

In [5]:
#Navigate dictionaries to available data formats
mngeo_resources = mngeo_data['result']['resources']

In [6]:
#Find the url of the shapefile
mngeo_shp_url = None
for resource in mngeo_resources:
    if resource['format'] == 'SHP':
        mngeo_shp_url = resource['url']
        break

In [7]:
#Write the data to a locally stored, zipped shapefile
mngeo_shp = requests.get(mngeo_shp_url)
with open('C:/Users/tjjoh/Documents/GIS5571/mn-counties.zip', 'wb') as file:
    file.write(mngeo_shp.content)

In [8]:
#Unzip the data
with zipfile.ZipFile('C:/Users/tjjoh/Documents/GIS5571/mn-counties.zip', 'r') as zip_ref:
    zip_ref.extractall('C:/Users/tjjoh/Documents/GIS5571/mn-counties')

In [9]:
#Convert data to a sedf
mn_counties_sedf = GeoAccessor.from_featureclass('C:/Users/tjjoh/Documents/GIS5571/mn-counties/mn_county_boundaries_multipart.shp')

In [10]:
mn_counties_sedf.spatial.project(SpatialReference(4326))

True

Load data from ArcGIS Server

In [35]:
#Data url
esri_url = 'https://webgis.dot.state.mn.us/65agsf1/rest/services/sdw_trans/AIRPORT/FeatureServer/0/query?where=1=1&f=geojson'

In [36]:
#Use ArcGIS REST API to call data
esri_response = requests.get(esri_url)

In [37]:
#Read the geojson file
airports = pandas.json_normalize(esri_response.json()['features'])

In [14]:
#Create new columns for latitude and longitude
airports['Longitude'] = airports['geometry.coordinates'].apply(lambda x: x[0])
airports['Latitude'] = airports['geometry.coordinates'].apply(lambda x: x[1])

In [15]:
#Convert data to a sedf
airports_sedf = GeoAccessor.from_xy(airports, x_column = 'Longitude', y_column = 'Latitude', sr = 4326)

Load data from NDAWN

In [16]:
#Data url. This can be copied from the NDAWN website, but "get-table.html" must be changed to "table.csv"
ndawn_url = 'https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=224&station=159&station=10&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=218&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=191&station=16&station=210&station=201&station=137&station=124&station=143&station=17&station=85&station=226&station=140&station=134&station=18&station=136&station=219&station=65&station=104&station=99&station=192&station=19&station=227&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=211&station=172&station=139&station=158&station=23&station=157&station=220&station=62&station=86&station=24&station=89&station=126&station=223&station=167&station=93&station=183&station=90&station=25&station=205&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=195&station=185&station=29&station=30&station=154&station=31&station=187&station=102&station=32&station=119&station=4&station=217&station=80&station=33&station=59&station=153&station=105&station=82&station=225&station=34&station=198&station=72&station=135&station=35&station=76&station=120&station=209&station=141&station=109&station=36&station=207&station=79&station=193&station=71&station=212&station=37&station=38&station=189&station=39&station=130&station=73&station=188&station=40&station=41&station=54&station=69&station=194&station=145&station=214&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=196&station=88&station=114&station=3&station=163&station=200&station=216&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=197&station=44&station=206&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=213&station=60&station=199&station=125&station=176&station=177&station=8&station=180&station=204&station=47&station=221&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=222&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=203&station=190&station=208&station=57&station=149&station=148&station=202&station=215&station=110&variable=ddmxt&year=2024&ttype=daily&quick_pick=&begin_date=2024-10-02&end_date=2024-10-02'

In [17]:
#read the .csv file using pandas. With indices starting at 0, data begins in row 5, with headers in row 3
#skipping rows 0, 1, 2, and 4 allows the data to be read correctly
nodak_weather = pandas.read_csv(ndawn_url, skiprows = [0,1,2,4])

In [18]:
#Convert data to a sedf
nodak_weather_sedf = GeoAccessor.from_xy(nodak_weather, x_column = 'Longitude', y_column = 'Latitude', sr = 4326)

Spatially join airports (from ArcGIS) and counties (from Minnesota Geospatial Commons)

In [19]:
#Spatially join data. Add county information to each airport
joined_data = airports_sedf.spatial.join(mn_counties_sedf, how = 'left', op = 'within')

In [20]:
#Remove extra columns that have geometry information
joined_data = joined_data.drop(columns = ['Shape_Leng', 'Shape_Area', 'geometry.coordinates'])

In [21]:
#View the data
joined_data.head()

,type,geometry.type,properties.FACILITYNAME,Longitude,Latitude,SHAPE,index_right,FID,COUNTYFIPS,COUNTYNAME
0,Feature,Point,FERGUS FALLS MUNI-EINAR MICKELSON FLD,-96.156703,46.284397,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96....",55.0,55,111,Otter Tail
1,Feature,Point,FERTILE MUNI,-96.292203,47.551925,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96....",59.0,59,119,Polk
2,Feature,Point,NORMAN COUNTY ADA/TWIN VALLEY,-96.400273,47.260476,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96....",53.0,53,107,Norman
3,Feature,Point,WALKER FIELD,-92.812530,44.931114,"{""spatialReference"": {""wkid"": 4326}, ""x"": -92....",81.0,81,163,Washington
4,Feature,Point,AITKIN MUNI-STEVE KURTZ FIELD,-93.676640,46.548470,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93....",0.0,0,001,Aitkin


Save data to a Geodatabase

In [22]:
#Create a file geodatabase
arcpy.management.CreateFileGDB(
    out_folder_path=r"C:/Users/tjjoh/Documents/GIS5571",
    out_name="Lab1_GDB",
    out_version="CURRENT"
)

<Result 'C:/Users/tjjoh/Documents/GIS5571/Lab1_GDB.gdb'>

In [23]:
#Specify the path to the geodatabase
gdb_path = r'C:/Users/tjjoh/Documents/GIS5571/Lab1_GDB.gdb'

In [24]:
#Send data to the geodatabase as feature classes
joined_data.spatial.to_featureclass(location = gdb_path + r'/Airport_County_Join')
nodak_weather_sedf.spatial.to_featureclass(location = gdb_path + r'/NoDak_Weather')

'C:\\Users\\tjjoh\\Documents\\GIS5571\\Lab1_GDB.gdb\\NoDak_Weather'